**LOAD LIBRARIES**

In [27]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import re
import json

import tweepy as tp
import twitter_credentials

# 1.2 DATA COLLECTION - MORE TWEETS FROM VERIFIED USERS

This notebooks uses tweepy to collect Tweets from a list of verified users in Twitter. This complements Notebook 1.1 and was generated after Milestone #3 submission.

**DEFINE USER LISTS**

In [28]:
# Define list of verified users for which to get tweets
v_user_list = ['@pnut',
               '@EYJr',
               '@MCRofficial',
               '@RM_Foundation',
               '@tomwaits',
               '@TherealTaraji',
               '@duttypaul',
               '@AvanJogia',
               '@AlecJRoss',
               '@s_vakarchuk',
               '@StephenColletti',
               '@jem',
               '@tonyparker',
               '@vitorbelfort',
               '@jeff_green22',
               '@johnwoodRTR',
               '@TomJackson57',
               '@robbiewilliams',
               '@AshleyMGreene',
               '@edhornick',
               '@mattdusk',
               '@ReggieEvans30',
               '@RachelNichols1',
               '@AndersFoghR',
               '@KAKA',
               '@Robbie_OC',
               '@josiahandthe',
               '@OKKenna',
               '@CP3',
               '@crystaltamar',
               '@MichelleDBeadle',
               '@AdamSchefter',
               '@georgelopez',
               '@CharlieDavies9',
               '@Nicole_Murphy',
               '@vkhosla',
               '@NathanPacheco',
               '@FreddtAdu',
               '@Erik_Seidel',
               '@Pamela_Brunson',
               '@MMRAW',
               '@russwest44',
               '@shawnieora',
               '@wingoz',
               '@ToddBrunson',
               '@NathanFillion',
               '@LaurenLondon',
               '@francescadani',
               '@SomethingToBurn',
               '@howardhlederer',
               '@MrBlackFrancis',
               '@GordonKljestan',
               '@thehitwoman',
               '@KeriHilson',
               '@kramergirl',
               '@druidDUDE',
               '@jimjonescapo',
               '@myfamolouslife',
               '@PAULVANDYK',
               '@SteveAustria',
               '@jensstoltenberg',
               '@bandofhoreses',
               '@jaysean',
               '@justdemi',
               '@MaryBonoUSA',
               '@PaulBrounMD',
               '@jrich23',
               '@Eve6',
               '@st_vincent',
               '@Padmasree',
               '@jamiecullum',
               '@GuyKawasaki',
               '@howardlindzon',
               '@xonecole',
               '@AlisonSudol',
               '@SuzyWelch',
               '@topchefkevin',
               '@MarcusCooks',
               '@Rick_Bayless',
               '@ShaniDavis',
               '@scottylago',
               '@TheRealSimonCho',
               '@ItsStephRice',
               '@IvanBabikov',
               '@DennyMdotcom',
               '@TFletchernordic',
               '@RockneBru86',
               '@JuliaMancuso',
               '@RyanOBedford',
               '@speedchick428',
               '@JennHeil',
               '@Jonnyboy77',
               '@katadamek',
               '@kathryn_kang',
               '@alejandrina_gr',
               '@RaymondArroyo',
               '@JonHaidt',
               '@StevePeers',
               '@DKShrewsbury',
               '@faisalislam',
               '@miqdaad',
               '@michikokakutani',
               '@mehdirhasan',
               '@AbiWilks',
               '@hugorifkind',
               '@kylegriffin1',
               '@timothy_stanley',
               '@NAXWELL',
               '@PT_Dawson',
               '@MaiaDunphy',
               '@danielralston',
               '@zachheltzel',
               '@KatyWellhousen',
               '@NicholasHoult',
               '@ryanbroems',
               '@LlamaGod',
               '@boozan',
               '@burbunny',
               '@DarrenMattocks',
               '@BraulioAmado',
               '@bernierobichaud',
               '@ThisisSIBA',
               '@Jill_Perkins3',
               '@crystalshawanda',
               '@D_Breitenstein',
               '@George_McD',
               '@RedAlurk',
               '@NickRobertson10',
               '@kevinvu',
               '@Henry_Kaye',
               '@Chris_Biele',
               '@tom_watson',
               '@MikeSegalov',
               '@edballs',
               '@TalbertSwan',
               '@eugenegu',
               '@PalmerReport',
               '@Weinsteinlaw',
               '@BrittMcHenry',
               '@ava',
               '@McFaul',
               '@DaShanneStokes',
               '@funder',
               '@Devonte_Riley',
               '@BrunoAmato_1',
               '@DirkBlocker',
               '@TrevDon',
               '@DavidYankovich',
               '@KirkDBorne',
               '@JohnLegere',
               '@PythonJones',
               '@JustinPollard',
               '@MattDudek',
               '@CoachWash56',
               '@RexxLifeRaj',
               '@SageRosenfels18',
               '@PollackHunter'
              ]
print(len(v_user_list))

157


**DEFINE MAX TWEETS PER USER**

In [29]:
# Define max number of tweets you want from each user, starting with most recentnum_tweets = 10
num_tweets = 150

**FUNCTIONS USED IN MAIN BODY**

In [30]:
def authenticate_twitter_app():
    # auth stuff
    consumer_key = twitter_credentials.CONSUMER_KEY 
    consumer_secret = twitter_credentials.CONSUMER_SECRET 
    auth = tp.OAuthHandler(consumer_key, consumer_secret)

    # token stuff
    access_token = twitter_credentials.ACCESS_TOKEN 
    access_token_secret = twitter_credentials.ACCESS_TOKEN_SECRET 
    auth.set_access_token(access_token, access_token_secret)

    return(auth)

def get_user_timeline_tweets(twitter_client, user_list, num_tweets):

    # this list will = tweeets & re-tweets (filter or tag re-tweets later)
    tweets = []

    for user in user_list:
        # This returns tweets & re-tweets, 10 at a time
        # We need to research more about getting larger volume back in time with max_id, since_id, etc.
        # I am hardcoding @realDonaldTrump...but we would put in variable from list of users
        print(f'Getting {num_tweets} tweets for {user}.')
        try:
            for tweet in tp.Cursor(twitter_client.user_timeline, id=user).items(num_tweets):
                tweets.append(tweet)    
        except tp.RateLimitError:
            print(f'SLEEPING DUE TO RATE LIMIT ERROR!!!!')
            time.sleep(15 * 60)
        except Exception as e:
            print(f'SOME ERROR OCCURRED...PASSING!!!')
            print(e.__doc__)
            pass
     

    return(tweets)
         
def produce_status_LoDs(statuses):
    
    tweet_LoD = []
    user_LoD = []
    for status in statuses:
        
        tweet_dict = {}
        user_dict = {}


        # tweet or retweet data 
        tweet_dict['user_id'] = status.author.id
        tweet_dict['user_screen_name'] = status.author.screen_name
        tweet_dict['created_at'] = status.created_at.isoformat()
        tweet_dict['id'] = status.id
        tweet_dict['id_str'] = status.id_str
        tweet_dict['text'] = status.text
        tweet_dict['source'] = status.source
        tweet_dict['truncated'] = status.truncated
 
        tweet_dict['retweet_count'] = status.retweet_count
        tweet_dict['favorite_count'] = status.favorite_count
        tweet_dict['lang'] = status.lang
        tweet_dict['is_tweet'] = ((re.search('RT', status.text) == None))
        ###Preimium API only?:  tweet_dict['retweeted_status'] = status.retweeted_status
        ###Preimium API only?:  tweet_dict['reply_count'] = status.reply_count
        ###Premium API only?:  tweet_dict['possibly_sensitive'] = status.possibly_sensitive
        
        tweet_LoD.append(tweet_dict)
           
                      
        ['created_at','description','favourites_count','followers_count','friends_count','id',
                      'lang','listed_count','location','name','screen_name','statuses_count',
                      'time_zone','verified'] 
                
        # user data
        user_dict['id'] = status.author.id
        #user_dict['id_str'] = status.author.id_str
        user_dict['name'] = status.author.name
        user_dict['screen_name'] = status.author.screen_name
        user_dict['location'] = status.author.location
        #user_dict['url'] = status.author.url
        user_dict['description'] = status.author.description
        user_dict['verified'] = status.author.verified
        user_dict['followers_count'] = status.author.followers_count
        user_dict['listed_count'] = status.author.listed_count
        user_dict['favourites_count'] = status.author.favourites_count
        user_dict['statuses_count'] = status.author.statuses_count
        user_dict['created_at'] = status.author.created_at.isoformat()
        #user_dict['utc_offset'] = status.author.utc_offset
        user_dict['time_zone'] = status.author.time_zone
        user_dict['lang'] = status.author.lang
        
        # Non-twitter, enriched field that dev team is manually managing
        if status.author.verified:
            user_dict['known_bot'] = False
        else:
            user_dict['known_bot'] = False # We will change to true when importing known bots

        user_LoD.append(user_dict)

        
    return(tweet_LoD, user_LoD)

**MAIN:  AUTHENTICATE**

In [31]:
# Setup twitter client to read data from specified user
auth = authenticate_twitter_app()
twitter_client = tp.API(auth)

**GET VERIFIED USER TWEETS FROM TWITTER**

In [32]:
# Get fixed number of tweets and put in results
statuses = get_user_timeline_tweets(twitter_client,v_user_list, num_tweets)

# Create list to write to json file like we did in HW1
tweet_LoD, user_LoD = produce_status_LoDs(statuses)

# Put in DF in case you skip the out/in via json below
v_tweet_df = pd.DataFrame(tweet_LoD)
v_user_df = pd.DataFrame(user_LoD)

Getting 150 tweets for @pnut.
Getting 150 tweets for @EYJr.
Getting 150 tweets for @MCRofficial.
Getting 150 tweets for @RM_Foundation.
Getting 150 tweets for @tomwaits.
Getting 150 tweets for @TherealTaraji.
Getting 150 tweets for @duttypaul.
Getting 150 tweets for @AvanJogia.
Getting 150 tweets for @AlecJRoss.
Getting 150 tweets for @s_vakarchuk.
Getting 150 tweets for @StephenColletti.
Getting 150 tweets for @jem.
Getting 150 tweets for @tonyparker.
Getting 150 tweets for @vitorbelfort.
Getting 150 tweets for @jeff_green22.
Getting 150 tweets for @johnwoodRTR.
Getting 150 tweets for @TomJackson57.
Getting 150 tweets for @robbiewilliams.
Getting 150 tweets for @AshleyMGreene.
Getting 150 tweets for @edhornick.
Getting 150 tweets for @mattdusk.
Getting 150 tweets for @ReggieEvans30.
Getting 150 tweets for @RachelNichols1.
Getting 150 tweets for @AndersFoghR.
Getting 150 tweets for @KAKA.
Getting 150 tweets for @Robbie_OC.
Getting 150 tweets for @josiahandthe.
Getting 150 tweets for @O

In [33]:
print(v_tweet_df.shape)
print(v_user_df.shape)

print(len(tweet_LoD))
print(len(user_LoD))

(16700, 12)
(16700, 14)
16700
16700


**WRITE VERIFIED TWEETS TO JSON FILE**

In [34]:
# write star table to disk as .json file
import json

with open('data/v_2nd_tweet_table_out.json', 'w') as outfile:  
    json.dump(tweet_LoD, outfile)
    

with open('data/v_2nd_user_table_out.json', 'w') as outfile:  
    json.dump(user_LoD, outfile)

In [35]:
################################
# Verified
################################
with open("data/v_2nd_tweet_table_out.json") as datafile:
    data = json.load(datafile)
v_tweet_df = pd.DataFrame(data)
v_tweet_df = v_tweet_df.drop_duplicates('id')

with open("data/v_2nd_user_table_out.json") as datafile:
    data = json.load(datafile)
v_user_df = pd.DataFrame(data)
v_user_df = v_user_df.drop_duplicates('id')


In [36]:
print(v_user_df.shape)
print(v_tweet_df.shape)

(112, 14)
(16700, 12)


In [37]:
user_df = v_user_df
tweet_df = v_tweet_df

In [38]:
print(user_df.shape)
print(tweet_df.shape)

(112, 14)
(16700, 12)


In [39]:
user_json = user_df.to_json(orient='records')
tweet_json = tweet_df.to_json(orient='records')

In [40]:
with open('data/more_final_tweet_master.json', 'w') as outfile:  
    json.dump(tweet_json, outfile)
    
with open('data/more_final_user_master.json', 'w') as outfile:  
    json.dump(user_json, outfile)

In [41]:
#with open('tweet_master.json') as json_file:  
with open('data/more_final_tweet_master.json') as json_file:  
    tweet_json = json.load(json_file)

#with open('user_master.json') as json_file:  
with open('data/more_final_user_master.json') as json_file:  
     user_json = json.load(json_file)

In [42]:
tweet_df = pd.read_json(tweet_json)

user_df = pd.read_json(user_json)

print(user_df.shape)
print(tweet_df.shape)

(112, 14)
(16700, 12)
